<a href="https://colab.research.google.com/github/kimolaevkamal/task_1_YOLOv8_Sobel/blob/main/Task_1_AlsihAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python-headless

In [ ]:
import cv2
import os
from pathlib import Path
from ultralytics import YOLO
import json

In [ ]:
dataset_path = Path("/content/drive/MyDrive/datasets1_task_1")  # замени на путь к папке с train2017/ и т.д.

train_dir = dataset_path / "train2017"
val_dir = dataset_path / "val2017"
test_dir = dataset_path / "test2017"

train_json = dataset_path / "instances_train2017.json"
val_json = dataset_path / "instances_val2017.json"
test_json = dataset_path / "instances_test2017.json"

In [ ]:
print(list(train_dir.glob("*.jpg")))

[PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001290.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001041.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001261.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001292.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001305.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001033.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001302.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001719.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/000999.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/000963.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/001275.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/000997.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train2017/000979.jpg'), PosixPath('/content/driv

In [ ]:
preproc_train = Path("/content/drive/MyDrive/datasets1_task_1/train_sobel")
print(list(preproc_train.glob("*.*")))

[PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001290.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001041.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001261.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001292.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001305.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001033.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001302.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001719.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/000999.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/000963.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/001275.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/000997.jpg'), PosixPath('/content/drive/MyDrive/datasets1_task_1/train_sobel/000979.jpg')

In [ ]:
def preprocess_image(img_path, save_path):
    img = cv2.imread(str(img_path))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobelx, sobely)
    sobel = cv2.normalize(sobel, None, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite(str(save_path), sobel)


preproc_train = dataset_path / "train_sobel"
preproc_val = dataset_path / "val_sobel"
preproc_train.mkdir(exist_ok=True)
preproc_val.mkdir(exist_ok=True)


for img_file in train_dir.glob("*.jpg"):
    preprocess_image(img_file, preproc_train / img_file.name)

for img_file in val_dir.glob("*.jpg"):
    preprocess_image(img_file, preproc_val / img_file.name)

In [ ]:
yolo_data_yaml = dataset_path / "my_dataset.yaml"
with open(yolo_data_yaml, "w") as f:
    f.write(f"""
train: {preproc_train}
val: {preproc_val}
test: {test_dir}
nc: 6
names: ['dent','scratch','crack','glass shatter','lamp broken','tire flat']
""")

In [ ]:
model = YOLO("yolov8n-seg.pt")  # можно заменить на yolov8s-seg.pt
model.train(
    data=str(yolo_data_yaml),
    imgsz=640,
    epochs=10,
    batch=8,
    name="yolov8_sobel_seg",
    project="runs"
)


Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/datasets1_task_1/my_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_sobel_seg3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patie

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



KeyboardInterrupt: 

In [ ]:
test_img = list(preproc_val.glob("*.jpg"))[0]
results = model.predict(source=str(test_img), show=True)

In [8]:
!git clone https://github.com/kimolaevkamal/task_1_YOLOv8_Sobel
%cd YOLOv8_Sobel

Cloning into 'task_1_YOLOv8_Sobel'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
[Errno 2] No such file or directory: 'YOLOv8_Sobel'
/content


In [10]:
!cp /content/Task_1_AlishAi.ipynb .

cp: cannot stat '/content/Task_1_AlishAi.ipynb': No such file or directory
